In [ ]:
!pip install webdataset
!pip install faiss-cpu

In [ ]:
import tarfile
import json
import os
import torch
from datasets import Dataset
import webdataset as wds
import hashlib
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = wds.WebDataset("/kaggle/input/ragdata/shard_aa/shard_aa").decode()

for i, sample in enumerate(dataset):
    print("\nSample", i)
    print("Keys:", sample.keys())
    if "txt" in sample:
        print("Text Preview:", sample["txt"][:400])  # first 400 chars
    if "json" in sample:
        print("JSON Preview:", str(sample["json"])[:400])
    if i == 3:   # stop after a few
        break

In [ ]:
shard_paths = [
    "/kaggle/input/ragdata/shard_aa/shard_aa",
    "//kaggle/input/ragdata/shard_ab/shard_ab",
    "//kaggle/input/ragdata/shard_ac/shard_ac",
    "//kaggle/input/ragdata/shard_ad/shard_ad",
    "//kaggle/input/ragdata/shard_ae/shard_ae",
]

dataset = wds.WebDataset(shard_paths).decode()

In [ ]:
MODEL_NAME = 'all-MiniLM-L6-v2'
CHUNK_SIZE = 256  # Max tokens for the embedding model
CHUNK_OVERLAP = 32 

print(f"\nLoading sentence-transformer model: '{MODEL_NAME}'...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer(MODEL_NAME, device=device)
print(f"Model loaded successfully on device: '{device}'")


# Initialize a text splitter to handle long documents.
# This ensures that no text passed to the model exceeds its token limit.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len, 
    is_separator_regex=False,
)

In [ ]:
embedded_chunks = []

# Iterate through each sample in the dataset
for sample in dataset:
    key = sample.get('__key__', 'unknown_key')
    text_content = ""

    # --- a. Extract Text Content ---
    # Determine the source of text and extract it.
    if 'txt' in sample:
        text_content = sample['txt']
    elif 'json' in sample:
        json_data = sample['json']
        if 'search_results' in json_data and isinstance(json_data.get('search_results'), list):
            # Concatenate snippets from search results
            snippets = [result.get('snippet', '') for result in json_data['search_results']]
            text_content = ' '.join(filter(None, snippets)) # Filter out empty snippets
        else:
            # As a fallback, serialize the entire JSON object to a string
            text_content = json.dumps(json_data)
    
    if not text_content or not text_content.strip():
        print(f"  - WARNING: Skipping sample '{key}' because it has no text content.")
        continue

    # --- b. Chunk the Text ---
    # The splitter will return a list with one item if the text is short,
    # or multiple chunks if it's long.
    chunks = text_splitter.split_text(text_content)
    print(f"  - Processing sample '{key}': Extracted text split into {len(chunks)} chunk(s).")

    # --- c. Generate Embeddings for Each Chunk ---
    # model.encode can process a list of texts at once, which is very efficient.
    chunk_embeddings = model.encode(chunks, convert_to_tensor=False) # Get numpy arrays

    # --- d. Store the Results ---
    for i, chunk in enumerate(chunks):
        embedded_chunks.append({
            'original_key': key,      # The key of the source document
            'chunk_index': i,         # The index of this chunk within the document
            'chunk_text': chunk,      # The actual text of the chunk
            'embedding': chunk_embeddings[i] # The embedding for this chunk
        })


  - Processing sample 'final_train/10306552__OQ__O-2006-0005__EN_e41489': Extracted text split into 16 chunk(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  - Processing sample 'final_train/059343-5107_doublejeopardy_moviequotes_400_bc2cf9': Extracted text split into 38 chunk(s).


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  - Processing sample 'final_train/29785339__WQA__E-2011-001330__EN': Extracted text split into 4 chunk(s).


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  - Processing sample 'final_train/085563-4237_doublejeopardy_foreignfilms_400_07ff39': Extracted text split into 137 chunk(s).


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

  - Processing sample 'final_train/7394975__WQ__P-1999-1024__EN_484ca5': Extracted text split into 3 chunk(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
print(f"\n--- Processing Complete ---")
print(f"Total number of chunks embedded: {len(embedded_chunks)}")

# Print details of the first 3 generated chunks for verification
print("\n--- Sample of Embedded Chunks ---")
for i, item in enumerate(embedded_chunks[:3]):
    print(f"\n[Chunk {i+1}]")
    print(f"  Original Key: {item['original_key']}")
    print(f"  Chunk Index:  {item['chunk_index']}")
    print(f"  Chunk Text:   '{item['chunk_text'][:100]}...'") # Preview of the text
    print(f"  Embedding Shape: {item['embedding'].shape}")


--- Processing Complete ---
Total number of chunks embedded: 2376512

--- Sample of Embedded Chunks ---

[Chunk 1]
  Original Key: final_train/21972291__WQ__P-2009-2118__EN
  Chunk Index:  0
  Chunk Text:   'WRITTEN QUESTION P-2118/09
by Witold Tomczak (IND/DEM)
to the Commission
(23 March 2009)
Subject: Pr...'
  Embedding Shape: (384,)

[Chunk 2]
  Original Key: final_train/21972291__WQ__P-2009-2118__EN
  Chunk Index:  1
  Chunk Text:   'In the above proposal, the Commission proposes inter alia a ban on the marketing of products from al...'
  Embedding Shape: (384,)

[Chunk 3]
  Original Key: final_train/21972291__WQ__P-2009-2118__EN
  Chunk Index:  2
  Chunk Text:   'of seals. In many cases, seals are pests posing a threat to fish stocks and their numbers needs to b...'
  Embedding Shape: (384,)


In [19]:
import pandas as pd
import pickle

In [20]:
print("\n--- Viewing and Saving Embeddings ---")

# --- Method 1: Viewing with Pandas DataFrame ---
# This is a great way to inspect your data in a structured table.
print("\nCreating Pandas DataFrame for easy viewing...")
df = pd.DataFrame(embedded_chunks)
# Display the first 5 rows of the dataframe.
# The 'embedding' column will show the numpy array.
print(df.head())



--- Viewing and Saving Embeddings ---

Creating Pandas DataFrame for easy viewing...
                                original_key  chunk_index  \
0  final_train/21972291__WQ__P-2009-2118__EN            0   
1  final_train/21972291__WQ__P-2009-2118__EN            1   
2  final_train/21972291__WQ__P-2009-2118__EN            2   
3  final_train/21972291__WQ__P-2009-2118__EN            3   
4  final_train/21972291__WQ__P-2009-2118__EN            4   

                                          chunk_text  \
0  WRITTEN QUESTION P-2118/09\nby Witold Tomczak ...   
1  In the above proposal, the Commission proposes...   
2  of seals. In many cases, seals are pests posin...   
3  hunts, which are subject to no controls whatso...   
4  poachers and might result in great suffering t...   

                                           embedding  
0  [-0.09138011, 0.030646732, -0.0027935628, -0.0...  
1  [-0.045915566, 0.03897162, 0.006841461, -0.047...  
2  [0.060897853, 0.07951124, 0.03683788, 0.03

In [21]:
# --- Method 2: Saving to Parquet (Recommended) ---
# Parquet is an efficient, column-oriented data format ideal for large datasets.
parquet_path = "/kaggle/working/embeddings.parquet"
print(f"\nSaving DataFrame to Parquet file: {parquet_path}")
df.to_parquet(parquet_path, index=False)
print("Saved successfully.")

# How to load it back:
# print("\nLoading data from Parquet file...")
# loaded_df = pd.read_parquet(parquet_path)
# print(loaded_df.head())


# --- Method 3: Saving to a Pickle file (Simple Alternative) ---
# Pickle is a Python-specific format that serializes the entire list object.
pickle_path = "/kaggle/working/embeddings.pkl"
print(f"\nSaving list to Pickle file: {pickle_path}")
with open(pickle_path, "wb") as f:
    pickle.dump(embedded_chunks, f)
print("Saved successfully.")

# How to load it back:
# print("\nLoading data from Pickle file...")
# with open(pickle_path, "rb") as f:
#     loaded_chunks = pickle.load(f)
# print(f"Loaded {len(loaded_chunks)} chunks from pickle.")


Saving DataFrame to Parquet file: /kaggle/working/embeddings.parquet
Saved successfully.

Saving list to Pickle file: /kaggle/working/embeddings.pkl
Saved successfully.


In [22]:
# How to load it back:
print("\nLoading data from Pickle file...")
with open('/kaggle/working/embeddings.pkl', "rb") as f:
    loaded_chunks = pickle.load(f)
print(f"Loaded {len(loaded_chunks)} chunks from pickle.")


Loading data from Pickle file...
Loaded 2376512 chunks from pickle.
